In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader

from fl_g13.config import RAW_DATA_DIR
from fl_g13.modeling import train, get_preprocessing_pipeline

from fl_g13.architectures import BaseDino
from fl_g13.editing import SparseSGDM

import dotenv

2025-06-24 10:12:57.013 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ciovi\Desktop\coding\mldl\fl-g13


In [3]:
dotenv.load_dotenv()

train_dataset, val_dataset, test_dataset = get_preprocessing_pipeline(RAW_DATA_DIR)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 40000
Validation dataset size: 10000
Test dataset size: 10000


In [4]:
# Move to CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Settings
name="azelf"
CHECKPOINT_DIR = dotenv.dotenv_values()["CHECKPOINT_DIR"]
start_epoch = 1
num_epochs = 30
save_every = 1
backup_every = None

# Model Hyper-parameters
head_layers = 3
head_hidden_size = 512
dropout_rate = 0.0
unfreeze_blocks = 12

# Training Hyper-parameters
batch_size = 128
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model
model = BaseDino(
    head_layers=head_layers, 
    head_hidden_size=head_hidden_size, 
    dropout_rate=dropout_rate, 
    unfreeze_blocks=unfreeze_blocks
    )
model.to(device)

# Optimizer, scheduler, and loss function
mask = [torch.ones_like(p, device=p.device) for p in model.parameters()] # Must be done AFTER the model is moved to CUDA
optimizer = SparseSGDM(
    model.parameters(),
    mask=mask,
    lr=lr,
    momentum=momentum,
    weight_decay=weight_decay
    )
scheduler = CosineAnnealingLR(
    optimizer=optimizer, 
    T_max=T_max, 
    eta_min=eta_min
    )
criterion = CrossEntropyLoss()

all_training_losses=[]       # Pre-allocated list for training losses
all_validation_losses=[]       # Pre-allocated list for validation losses
all_training_accuracies=[]    # Pre-allocated list for training accuracies
all_validation_accuracies=[]    # Pre-allocated list for validation accuracies

Using device: cuda


Using cache found in C:\Users\ciovi/.cache\torch\hub\facebookresearch_dino_main


In [5]:
try:
    _, _, _, _ = train(
        checkpoint_dir=f'{CHECKPOINT_DIR}/{name}',
        name=name,
        start_epoch=start_epoch,
        num_epochs=num_epochs,
        save_every=save_every,
        backup_every=backup_every,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        verbose=1,
        all_training_losses=all_training_losses,
        all_validation_losses=all_validation_losses,
        all_training_accuracies=all_training_accuracies,
        all_validation_accuracies=all_validation_accuracies,
        with_model_dir = False
    )

except KeyboardInterrupt:
    print("Training interrupted manually.")

except Exception as e:
    print(f"Training stopped due to error: {e}")

Prefix/name for the model was provided: azelf


Training progress: 100%|██████████| 313/313 [02:46<00:00,  1.88batch/s]


🚀 Epoch 1/30 (3.33%) Completed
	📊 Training Loss: 3.3008
	✅ Training Accuracy: 23.04%
	⏳ Elapsed Time: 166.33s | ETA: 4823.52s
	🕒 Completed At: 10:15


Eval progress: 100%|██████████| 79/79 [00:25<00:00,  3.13batch/s]


🔍 Validation Results:
	📉 Validation Loss: 1.5595
	🎯 Validation Accuracy: 55.19%
💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/azelf\azelf_BaseDino_epoch_1.pth
💾 Saved losses and accuracies (training and validation) at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/azelf\azelf_BaseDino_epoch_1.loss_acc.json



...

🔍 Validation Results:
	📉 Validation Loss: 0.6460
	🎯 Validation Accuracy: 85.51%
💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/azelf\azelf_BaseDino_epoch_29.pth
💾 Saved losses and accuracies (training and validation) at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/azelf\azelf_BaseDino_epoch_29.loss_acc.json



Training progress: 100%|██████████| 313/313 [02:43<00:00,  1.91batch/s]


🚀 Epoch 30/30 (100.00%) Completed
	📊 Training Loss: 0.0042
	✅ Training Accuracy: 99.92%
	⏳ Elapsed Time: 163.74s | ETA: 0.00s
	🕒 Completed At: 11:47


Eval progress: 100%|██████████| 79/79 [00:24<00:00,  3.20batch/s]


🔍 Validation Results:
	📉 Validation Loss: 0.6648
	🎯 Validation Accuracy: 85.28%
💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/azelf\azelf_BaseDino_epoch_30.pth
💾 Saved losses and accuracies (training and validation) at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/azelf\azelf_BaseDino_epoch_30.loss_acc.json

